# Imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from geomloss import SamplesLoss
import NCA.utils as utils
from NCA.NCA import *
from NCA.utils import  natural_keys
import csv
import os

### Whether Graphs Have a Normal Scale or Log Scale 

In [ ]:
NORMAL = True

# Graph for Final Training Loss Compared to Masking Level

In [ ]:
path = "Saved_Data/Butterfly"  #<--- Make sure its correct 
final_log = []
regrowth_log = []
# plt the min loss for each model compared to masking
x_axis = [0,1,2,3,4,5,6,7,8,9,10,11,12]
colors = ["r", "g", "b"]
ncas = ["Reduced", "Pre-Conv Artifact", "Post-Conv Artifact"]
plt.figure(4)

for id, (root, subdirs, files) in enumerate(os.walk(path, topdown=True)):
    means = []
    stds=[]
    if len(files) != 0:
        files.sort(key=natural_keys)
        for idx, file in enumerate(files):
            data = np.load(root+ "/" +file)
            if NORMAL:
                data1 = np.mean(np.exp(np.min(data)))/ 50**2
            else:
                data1 = np.min(data)
            data2 = np.std(data[-1:])
            print(file)
            means.append(data1)
            stds.append(data2)
        means = np.asarray(means)
        final_log.append(means)
        stds = np.asarray(stds)
        plt.plot(x_axis, means, color = colors[id-1], label=ncas[id-1])
        plt.ylabel('min(mean final loss)')
        plt.xlabel('N masked layers')
        plt.legend()
        plt.grid(True)

# Graph for Mean Regrowth Loss Compared to Masking Level 

In [ ]:
path = "Regrow_exp_data/Butterfly"
x_axis = [0,1,2,3,4,5,6,7,8,9,10,11,12]
colors = ["b", "g", "r"]
ncas = [ "Post-Conv Artifact", "Pre-Conv Artifact", "Reduced"]
plt.figure(3)
for id, (root, subdirs, files) in enumerate(os.walk(path, topdown=True)):
    if len(files) != 0:
        for idx, file in enumerate(files):
            data = np.load(root+ "/" +file)
            if NORMAL:
                data1 = np.exp(np.mean(data, axis=1))/ 50**2
            else:
                data1 = np.mean(data, axis=1)
            data2 = np.std(data, axis=1)
            regrowth_log.insert(0, data1)
            print(file )
            plt.plot(x_axis, data1, color = colors[idx], label=ncas[idx])
            plt.ylabel('mean loss after regrowth')
            plt.xlabel('N masked layers')
            plt.legend()
            plt.grid(True)


# Produces the CSV Comparing NCA through Gram Loss 

In [ ]:
path_masked = "Trained_models/Lizard/Masked" #<--- first set of models to compare 
path_dummy = "Trained_models/Lizard/Masked" #<--- second set of models to compare, can be same 
dummy_files = os.listdir(path_dummy)
masked_files = os.listdir(path_masked)
masked_files.sort(key=natural_keys)
dummy_files.sort(key=natural_keys)
mat = []
loss_fn = SamplesLoss("sinkhorn", p=2, blur=0.005)
for df in dummy_files:
    df = path_dummy + "/" + df
    nca1 = DummyVCA(16, 96, 0)
    nca1.load_state_dict(torch.load(df))
    nca1.cuda().eval()
    sum_losses = []
    for mf in masked_files:
        mf = path_masked + "/" + mf
        nca2 = DummyVCA(16,96,0)
        nca2.load_state_dict(torch.load(mf))
        nca2.cuda().eval()
        loss = 0
        for (name1,param1), (name2,param2) in zip(nca1.named_parameters(), nca2.named_parameters()):
            if len(param1.shape) >1:
                loss = loss + torch.nn.functional.mse_loss(utils.gram_matrix(param1.data), utils.gram_matrix(param2.data)).item()

        sum_losses.append(loss)
    mat.append(sum_losses)

# Creates the Path and Saves CSV

In [ ]:
csv_path = "CSVs"
if not os.path.exists(csv_path):
    os.makedirs(csv_path)
    print(f"Path: {csv_path} created")
else: 
    print(f"Path: {csv_path} already exists, all OK!")
    
    
file_path = "/inter_image_gram.csv" #<--- name of csv 
with open(csv_path+file_path, "w+") as my_csv:
    csvWriter = csv.writer(my_csv, delimiter=',')
    csvWriter.writerows(mat)

# Pearson Correlation Coef Between Regrowth Loss and Training Loss 

In [ ]:
for final_l, regrowth_log in zip(final_log, regrowth_log):
    print(np.corrcoef(final_l, regrowth_log))